In [41]:
# importing libraries

import os
import cv2
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Dense, Activation, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, Add, ZeroPadding2D,Flatten
import keras.backend as K
from keras.initializers import glorot_uniform
from keras.models import Model, load_model

In [42]:
# image parameters:

WIDTH = 128
HEIGHT = 128
CHANNELS = 3
CLASSES = 2

In [47]:
# implementation of Identity block

def identity_block(X, f, filters):

    # filters
    F1, F2, F3 = filters

    # input init

    X_shortcut = X

    #Identity block1

    X = Conv2D(filters = F1, kernel_size=(1,1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    #Identity block2

    X = Conv2D(filters = F2, kernel_size=(f,f), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    #Identity block3

    X = Conv2D(filters = F3, kernel_size=(1,1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)

    # Adding the X_shortcut to the main network

    X = Add()([X,X_shortcut])
    X  =Activation('relu')(X)

    return X



In [51]:
def Conv_block2D(X,f,filters,s = 2):

    # filters
    F1, F2, F3 = filters

    # inputs

    X_shortcut = X

    # Conv Block 1

    X = Conv2D(F1, kernel_size=(1, 1), strides = (s,s))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Conv Block 2

    X = Conv2D(F2, kernel_size = (f, f), strides = (1,1), padding='same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Conv Block 3

    X = Conv2D(F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)

    # Shortcut path
    X_shortcut = Conv2D(F3, kernel_size = (1, 1), strides = (s,s))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X




    


In [54]:

def ResNet50(input_shape = (128, 128, 3), classes = 2):   
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = Conv_block2D(X, f = 3, filters = [64, 64, 256],s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    # Stage 3
    X = Conv_block2D(X, f = 3, filters = [128, 128, 512], s = 2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4
    X = Conv_block2D(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5
    X = Conv_block2D(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL.
    X = AveragePooling2D((2, 2))(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid')(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [55]:
model = ResNet50(input_shape = (WIDTH, HEIGHT, CHANNELS), classes = CLASSES)

In [56]:
print(model.summary())

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_9 (ZeroPadding2  (None, 134, 134, 3)  0          ['input_11[0][0]']               
 D)                                                                                               
                                                                                                  
 conv1 (Conv2D)                 (None, 64, 64, 64)   9472        ['zero_padding2d_9[0][0]']       
                                                                                           